In [5]:
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *
# import PyQt5 #works

In [6]:
from scipy.signal import argrelextrema, argrelmin, find_peaks

In [7]:
# import glob
from matplotlib import cm

In [8]:
%matplotlib

Using matplotlib backend: TkAgg


In [9]:
# %matplotlib qt
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 20})

In [10]:
def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    C1 = q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a ) ) ) ) )
    C2 = q**2/( (beta_guess-2)**2 - a - q**2/( (beta_guess-4)**2 - a - q**2/( (beta_guess-6)**2 - a - q**2/( (beta_guess-8)**2 - a - q**2/( (beta_guess-10)**2 - a  ) ) ) ) )
    return a + C1 + C2

In [11]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide


r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = (4e-3)/2            # longueur piège Ca+
Omega = 2.045e6*2*pi # 2.047e6
bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
mkappa = 0.23
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [12]:
# Computation of secular frequencies

# set voltages
Udc = 0.1 # 2.8628*4   9.014464
Urf = 110/4 #42.43 # q*(m_Ca*r0**2*Omega**2)/( 4*C_e)
q = ( (m_Ca*r0**2*Omega**2)/( 4*C_e)/Urf )**(-1)
print('Urf =',Urf)

# compute beta
beta_guess = 0.423
a = 0
for k in range(15):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))

# compute angular frequencies
omega_z_2 = 2*C_e*(mkappa)*Udc/m_Ca/d0**2
# omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

# compute Mathieu parameters
qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
ax = 8*C_e*0/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_x_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)

Urf = 27.5
non adiab 0.18387332660003555
adiab     0.18145061219645256
alpha = 0.01983723475764424
qx = 0.25660991666912414
ax = 0.0
az = 0.00033534250473806
f_z = 26480.2826493335
f_x = 188010.47644853636
f_r = 187075.7508866698


# Extraction de données

In [13]:
filename=[]
# K:/20220713/tickle_169.9Vpp_2500mV_0_0_100k_400k_000.dat

In [14]:
# sélection des fichiers
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(initialdir = '/home/adrian/Documents/Mesures',multiple=True)

In [17]:
file_path

('/home/adrien/Documents/Mesures/GiantMol_Ions/220719_all/20220719_PM/tickle_110Vpp_300_50_1800mV_0_100kHz_000.dat',
 '/home/adrien/Documents/Mesures/GiantMol_Ions/220719_all/20220719_PM/tickle_110Vpp_300_50_1800mV_0_100kHz_001.dat',
 '/home/adrien/Documents/Mesures/GiantMol_Ions/220719_all/20220719_PM/tickle_110Vpp_300_50_1800mV_0_100kHz_002.dat',
 '/home/adrien/Documents/Mesures/GiantMol_Ions/220719_all/20220719_PM/tickle_110Vpp_300_50_1800mV_0_100kHz_003.dat',
 '/home/adrien/Documents/Mesures/GiantMol_Ions/220719_all/20220719_PM/tickle_110Vpp_300_50_1800mV_0_100kHz_004.dat',
 '/home/adrien/Documents/Mesures/GiantMol_Ions/220719_all/20220719_PM/tickle_110Vpp_300_50_1800mV_0_250kHz_000.dat')

In [15]:
# open the files and save the data
# TP is time
# PM is fluo
# VB is voltage applied to the GBF
filename=file_path
m=len(filename)
n=len(loadtxt('%s' %filename[1],usecols=(0),unpack=True))
shape=(m,n)
# Temps, PM, Tension Pzt bleu
TP = zeros(shape)
PM = zeros(shape)
VB = zeros(shape)
for k in range(0,len(filename),1):
    TP[:][k],PM[:][k],VB[:][k] = loadtxt('%s' %filename[k],usecols=(0,1,3),unpack=True)   

In [ ]:
pos_freq

In [25]:
# chose a file and plot it
to_plot = 3
# pos_freq = file_path[to_plot].find('0_0_')
pos_freq = file_path[to_plot].find('mV_')
freq_start = int(file_path[to_plot][pos_freq+3:pos_freq+4])
freq_end   = int(file_path[to_plot][pos_freq+5:pos_freq+8])
freq = linspace(freq_start,freq_end,len(TP[:][to_plot]))
# print(freq)
#pos_min = find_peaks(PM[:][to_plot],prominence=(250,None))[0] # max(PM[:][to_plot])/2,distance=50
pos_min = argrelextrema(PM[:][to_plot],less,order=50)

figure('Plot tickle').clf()

plot(freq,PM[to_plot],'')
plot(freq[pos_min],PM[to_plot][pos_min],'P') # [800]*len(freq[pos_min])
# plot(freq[1000-2:-1],PM[1],'r')
xlabel('Tickle frequency [kHz]')
ylabel('Fluorescence [cnt/250ms]')
grid()
plt.tight_layout()

In [ ]:
# bad start
# 0 1
# bad resonance
# 2 3     4 5 6 7 24 bad also

# exploitable
# 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22? 23?

In [ ]:
argrelextrema(PM_170_2000,less,order=50)[0]

In [ ]:
# let's compare
# 8 9 10           tickle_170Vpp_2000mV_0_0_100k_400k
PM_170_2000 = mean(PM[:][8:11],axis=0)
freq_170_2000 = linspace(100,400,len(TP[:][13]))
pos_min_170_2000 = argrelextrema(PM_170_2000,less,order=80)[0]
# 13 14 15 16 17   tickle_170Vpp_1000mV_0_0_050k_350k
PM_170_1000 = mean(PM[:][13:18],axis=0)
freq_170_1000 = linspace(50,350,len(TP[:][13]))
pos_min_170_1000 = argrelextrema(PM_170_1000,less,order=80)[0]

In [ ]:
figure('Plot tickle 170_2000').clf()

plot(freq_170_2000,PM_170_2000)
plot(freq_170_2000[pos_min_170_2000],PM_170_2000[pos_min_170_2000],'P')

for i,j in enumerate(pos_min_170_2000):
    text(freq_170_2000[j],PM_170_2000[j]*0.975,f'{freq_170_2000[j]:.1f}',fontsize=12)
# plot(freq_170_1000,PM[:][10])
# plot(freq[pos_min],PM[to_plot][pos_min],'P') # [800]*len(freq[pos_min])
# plot(freq[1000-2:-1],PM[1],'r')
xlabel('Tickle frequency [kHz]')
ylabel('Fluorescence [cnt/60ms]')
grid()
plt.tight_layout()

In [ ]:
# plot freq of peaks
print(freq_170_1000[pos_min_170_2000])

# compare with expecteed values
# for Urf = 42.43 V, Udc = 2 V
# f_z = 118423.42413463586
# f_x = 295882.3513634161
# f_r = 283785.679970745

In [ ]:
figure('Plot tickle 170_1000').clf()

plot(freq_170_1000,PM_170_1000)
plot(freq_170_1000[pos_min_170_1000],PM_170_1000[pos_min_170_1000],'P')

for i,j in enumerate(pos_min_170_1000):
    text(freq_170_1000[j],PM_170_1000[j]*0.975,f'{freq_170_1000[j]:.1f}',fontsize=12)
# plot(freq_170_1000,PM[:][10])
# plot(freq[pos_min],PM[to_plot][pos_min],'P') # [800]*len(freq[pos_min])
# plot(freq[1000-2:-1],PM[1],'r')
xlabel('Tickle frequency [kHz]')
ylabel('Fluorescence [cnt/60ms]')
grid()
plt.tight_layout()

In [ ]:
# plot freq of peaks
print(freq_170_1000[pos_min_170_1000])

# compare with expecteed values
# for Urf = 42.43 V, Udc = 1 V
# f_z = 83738.00625693168
# f_x = 295882.3513634161
# f_r = 289897.1179615297

In [ ]:
# data from 220720
# let's compare
# 8 9 10           tickle_170Vpp_2000mV_0_0_100k_400k
PM_mean = mean(PM[:][0:2],axis=0)
freq = linspace(100,400,len(TP[:][13]))
pos_min_170_2000 = argrelextrema(PM_mean,less,order=80)[0]